In [ ]:
# #PREREQUISITES FOR COLAB

# !git clone https://github.com/uniquetrij/action-detection.git

# import os
# os.chdir("action-detection/")

# !pwd

# !git pull


In [ ]:

from dataset import DataLoader, LabeledPathListingGenerator, DataSet
from matplotlib import pyplot as plt

from keras.utils import np_utils, generic_utils
from keras.layers.convolutional import Conv3D
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Activation, Dropout
from keras.layers.convolutional import Convolution3D
from keras.layers.pooling import MaxPooling3D


In [ ]:
data_loader = DataLoader("video_label.pkl", "label_encoding.pkl")
train_set, val_set, test_set = data_loader.generate_split([0.7,0.2,0.1])

In [ ]:
def build_model(class_count, frame_count, width, height):
    model = Sequential()

    model.add(Conv3D(32, kernel_size=(3, 7, 7), input_shape=(frame_count, height, width, 1), activation='relu'))
    model.add(MaxPooling3D(pool_size=(1, 3, 3)))
    model.add(Dropout(0.75))

    model.add(Conv3D(64, kernel_size=(3, 11, 11), activation='relu' ))
    model.add(Dropout(0.75))
    
    model.add(Conv3D(64, kernel_size=(5 ,11, 11), activation='relu' ))
#     model.add(MaxPooling3D(pool_size=(3, 3, 3)))
    model.add(Dropout(0.5))
    
    model.add(Conv3D(64, kernel_size=(5, 11, 11), activation='relu' ))
    model.add(Dropout(0.5))

    model.add(Flatten())

    model.add(Dense(64, init='normal', activation='relu'))
    model.add(Dropout(0.75))

    model.add(Dense(class_count, init='normal'))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    model.summary()

    return model;


In [ ]:
frame_count = 15
batch_size = 10
class_count = 2
model = build_model(class_count, frame_count, train_set.width, train_set.height)


In [ ]:
hist = model.fit_generator(train_set.get_batch_generator(batch_size,frame_count),
                       validation_data=val_set.get_batch_generator(batch_size,frame_count),
                       epochs=10,
                              steps_per_epoch=train_set.size/batch_size,
                              validation_steps=val_set.size/batch_size)


